In [1]:
options(warn = -1)

In [2]:
library(lubridate)
library(rgdal)
library(stringr)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: sp

rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj



In [3]:
data <- read.csv('T:/Tableau/tableauBikesOnBuses/Datasources/MonthlyBikesOnBuses.csv')

In [4]:
data$date <- as.Date(data$date, "%Y-%m-%d")

In [5]:
data$year <- year(data$date)

In [6]:
dat <- data[data$year == 2021,]

In [7]:
names(dat)

[1] "stop"      "srv"       "date"      "block"     "trip_end"  "time"     
 [7] "route"     "dir"       "stop_name" "bus"       "odometer"  "desc"     
[13] "qty"       "MonthYear" "longitude" "latitude"  "year"

In [8]:
convert_time_to_hour <- function(x){
    x <- unlist(str_split(x, " "))[2]
    s <- unlist(str_split(x, ":"))
    return(as.numeric(s[1])+as.numeric(s[2])/60)
}

In [9]:
dat$hour <- sapply(dat$trip_end, function(x) convert_time_to_hour(x))

In [10]:
get_parts_of_day <- function(x){
    if(x >= 21 | x <= 4){
        d <- "Night"
    }else if(x > 17 & x < 21){
        d <- "Evening"
    }else if(x > 12 & x <= 17){
        d <- "Afternoon"
    }else{
        d <- "Morning"
    }
    return(d)
}

In [11]:
dat$DayPart <- sapply(dat$hour, function(x) get_parts_of_day(x))

In [12]:
unique(dat$dir)

[1] "O" "I"

In [13]:
agg_data <- function(df=dat, by="I"){
    dt <- df[df$dir == by,]
    aggdt <- aggregate(x=list(qty=dt$qty),
              by=list(stop = dt$stop, stop_name = dt$stop_name, route=dt$route, longitude = dt$longitude, latitude=dt$latitude, daypart=dt$DayPart),
              FUN=sum)
    return(aggdt)
}

In [14]:
inbound_dt <- agg_data()

In [22]:
head(inbound_dt)

,stop,stop_name,route,longitude,latitude,daypart,qty
,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>
1,9800,E/S of WalMart Rd N of Thomas Ln,98,-123.0406,43.79921,Afternoon,64
2,9821,E/S of Row River N of Thornton,98,-123.0403,43.80244,Afternoon,1
3,9830,N/S of C E of 1st,98,-123.0198,43.91701,Afternoon,92
4,9832,S/S of E. Oregon Ave. E of Hwy 99,98,-123.0169,43.91781,Afternoon,3
5,9833,N/S of E. Oregon Ave. E of Hwy 99,98,-123.0173,43.91794,Afternoon,4
6,516,E/S of Donald S/O Cul deSac 5315-35,24,-123.0906,43.99823,Afternoon,7


In [15]:
outbound_dt <- agg_data(by="O")

In [16]:
df2spdf <- function(df, lon_col_name, lat_col_name, trans = TRUE){
  lonlat <- sp::CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
  lon_col_no <- which(names(df)==lon_col_name)
  lat_col_no <- which(names(df)==lat_col_name)
  xy <- data.frame(df[,c(lon_col_no,lat_col_no)])
  coordinates(xy) <- c(lon_col_name, lat_col_name)
  proj4string(xy) <- lonlat
  spdf <- sp::SpatialPointsDataFrame(coords = xy, data = df)
  if(trans){
    spdf <- spTransform(spdf,  CRS("+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs +type=crs"))
  }
  return(spdf)
}

In [17]:
inbound_spdf <- df2spdf(inbound_dt, 'longitude', 'latitude')

In [23]:
head(inbound_spdf@data)

,stop,stop_name,route,longitude,latitude,daypart,qty
,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>
1,9800,E/S of WalMart Rd N of Thomas Ln,98,-123.0406,43.79921,Afternoon,64
2,9821,E/S of Row River N of Thornton,98,-123.0403,43.80244,Afternoon,1
3,9830,N/S of C E of 1st,98,-123.0198,43.91701,Afternoon,92
4,9832,S/S of E. Oregon Ave. E of Hwy 99,98,-123.0169,43.91781,Afternoon,3
5,9833,N/S of E. Oregon Ave. E of Hwy 99,98,-123.0173,43.91794,Afternoon,4
6,516,E/S of Donald S/O Cul deSac 5315-35,24,-123.0906,43.99823,Afternoon,7


In [18]:
outbound_spdf <- df2spdf(outbound_dt, 'longitude', 'latitude')

In [19]:
path <- "T:/DCProjects/StoryMap/BikeCounting/BikeOnBuses/Output"

In [20]:
writeOGR(inbound_spdf, dsn=path, layer="Inbound_BOB_DayPart", driver="ESRI Shapefile", overwrite_layer=TRUE)

In [21]:
writeOGR(outbound_spdf, dsn=path, layer="Outbound_BOB_DayPart", driver="ESRI Shapefile", overwrite_layer=TRUE)